In [12]:
import os

os.environ["EB_AGENT_LOGGING_LEVEL"] = "INFO"
os.environ["EB_AGENT_ACCESS_TOKEN"] = "4ce50e3378f418d271c480c8ddfa818537071dbe"

from pydantic import Field
from typing import Dict, Type, List, Any

from erniebot_agent.chat_models import ERNIEBot
from erniebot_agent.memory import HumanMessage, Message, AIMessage, FunctionMessage, SlidingWindowMemory
from erniebot_agent.tools.base import Tool
from erniebot_agent.tools.schema import ToolParameterView
from erniebot_agent.agents import FunctionAgent
from erniebot_agent.tools import RemoteToolkit
from erniebot_agent.file import GlobalFileManagerHandler

In [15]:
import os
from erniebot_agent.agents.function_agent_with_retrieval import FunctionAgentWithRetrieval
from erniebot_agent.memory.whole_memory import WholeMemory
from erniebot_agent.chat_models.erniebot import ERNIEBot
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import UnstructuredMarkdownLoader
from erniebot_agent.extensions.langchain.embeddings import ErnieEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
from erniebot_agent.tools import RemoteToolkit

embeddings = ErnieEmbeddings(aistudio_access_token=os.environ["EB_AGENT_ACCESS_TOKEN"], chunk_size=16)


class FaissSearch:
    def __init__(self, db, embeddings):
        # 类的初始化方法，接收一个数据库实例并将其存储在类的实例变量 self.db 中，接收一个embeddings方法传到self.embeddings中
        self.db = db
        self.embeddings = embeddings

    def search(self, query: str, top_k: int = 10, **kwargs):
        # 定义一个搜索方法，接受一个查询字符串 'query' 和一个整数 'top_k'，默认为 10
        docs = self.db.similarity_search(query, top_k)
        # 调用数据库的 similarity_search 方法来获取与查询最相关的文档
        para_result = self.embeddings.embed_documents([i.page_content for i in docs])
        # 对获取的文档内容进行嵌入（embedding），以便进行相似性比较
        query_result = self.embeddings.embed_query(query)
        # 对查询字符串也进行嵌入
        similarities = cosine_similarity([query_result], para_result).reshape((-1,))
        # 计算查询嵌入和文档嵌入之间的余弦相似度
        retrieval_results = []
        for index, doc in enumerate(docs):
            retrieval_results.append(
                {"content": doc.page_content, "score": similarities[index], "title": doc.metadata["source"]}
            )
        # 遍历每个文档，将内容、相似度得分和来源标题作为字典添加到结果列表中
        return retrieval_results  # 返回包含搜索结果的列表

In [36]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
]


md_file_path = ["../../docs/modules/file.md", "../../docs/modules/agent.md"]


def read_md_file(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            md_content = file.read()
        return md_content
    except FileNotFoundError:
        print(f"文件 '{file_path}' 不存在。")
        return None
    except Exception as e:
        print(f"读取文件时出现错误： {e}")
        return None


# 例子
md_file_path = "/Users/tanzhehao/Documents/ERINE/ERNIE-Bot-SDK/docs/modules/file.md"
content = read_md_file(md_file_path)


markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(content)

from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 500
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# Split
splits = text_splitter.split_documents(md_header_splits)

db = FAISS.from_documents(splits, embeddings)

In [35]:
def handle

'/Users/tanzhehao/Documents/ERINE/ERNIE-Bot-SDK/erniebot-agent/applications'

In [38]:
res = db.similarity_search(query="如何使用File类", top_k=5)

In [ ]:
similarities = cosine_similarity([query_result], para_result).reshape((-1,))

In [41]:
res[0]

Document(page_content='`LocalFile` 是 `File` 的子类，表示本地文件。除了继承自基类的属性外，它还添加了文件路径属性 `path`，用于表示文件在本地文件系统中的路径。', metadata={'Header 1': 'File 模块', 'Header 2': '2. 核心类', 'Header 3': '2.2 File 子类', 'Header 4': '2.2.1 LocalFile 类'})

In [21]:
md_header_splits

[Document(page_content='文件管理模块提供了用于管理文件的一系列类，方便用户与Agent进行交互，其中包括 `File` 基类及其子类、`FileManager` 、`GlobalFileManagerHandler`以及与远程文件服务器交互的  `RemoteFileClient`。  \n推荐使用  `GlobalFileManagerHandler`在事件循环开始时初始化 `FileManager`以及获取全局的 `FileManager`，之后只需通过这个全局的 `FileManager`对文件进行增、删、查等操作以及获取Agent产生的文件。  \n!!! notes 注意  \n- **不推荐**用户自行操作 `File`类以免造成资源泄露。  \n- `FileManager`操作文件主要用于异步函数中，在同步函数中使用可能会无效。  \n- `FileManager`将作为此模块中生命周期最长的对象，它会在关闭时回收所有的持有对象（RemoteClient/temp local file），请不要随意关闭它。如果需要关闭，需要先停止对其中注册文件的使用。', metadata={'Header 1': 'File 模块', 'Header 2': '1. 模块简介'}),
 Document(page_content='下面简单介绍 `File` 模块的核心类，详细接口请参考[API文档](../package/erniebot_agent/file.md)。', metadata={'Header 1': 'File 模块', 'Header 2': '2. 核心类'}),
 Document(page_content='`File` 类是文件管理模块的基础类，用于表示通用的文件对象（不建议自行创建 `File` 类以免无法被 `Agent`识别使用以及无法被回收）。它包含文件的基本属性，如文件ID、文件名、文件大小、创建时间、文件用途和文件元数据。  \n此外， `File`类还定义了一系列抽象方法，比较常用的有：  \n* 异步读取文件内容的 `read_contents`方法\n* 将文件内容写入本地路径的 `write_contents_to`方法  \n以及一些辅助方法：  \n* 生成文件的字符串表示形式\n* 转

In [19]:
documents

In [9]:
def read_md_file(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            md_content = file.read()
        return md_content
    except FileNotFoundError:
        print(f"文件 '{file_path}' 不存在。")
        return None
    except Exception as e:
        print(f"读取文件时出现错误： {e}")
        return None


# 例子
md_file_path = "/Users/tanzhehao/Documents/ERINE/ERNIE-Bot-SDK/docs/modules/file.md"
content = read_md_file(md_file_path)

if content:
    print(content)

# File 模块

## 1. 模块简介

文件管理模块提供了用于管理文件的一系列类，方便用户与Agent进行交互，其中包括 `File` 基类及其子类、`FileManager` 、`GlobalFileManagerHandler`以及与远程文件服务器交互的  `RemoteFileClient`。

推荐使用  `GlobalFileManagerHandler`在事件循环开始时初始化 `FileManager`以及获取全局的 `FileManager`，之后只需通过这个全局的 `FileManager`对文件进行增、删、查等操作以及获取Agent产生的文件。

!!! notes 注意

    - **不推荐**用户自行操作 `File`类以免造成资源泄露。

    - `FileManager`操作文件主要用于异步函数中，在同步函数中使用可能会无效。

    - `FileManager`将作为此模块中生命周期最长的对象，它会在关闭时回收所有的持有对象（RemoteClient/temp local file），请不要随意关闭它。如果需要关闭，需要先停止对其中注册文件的使用。

## 2. 核心类

下面简单介绍 `File` 模块的核心类，详细接口请参考[API文档](../package/erniebot_agent/file.md)。

### 2.1 File 基类

`File` 类是文件管理模块的基础类，用于表示通用的文件对象（不建议自行创建 `File` 类以免无法被 `Agent`识别使用以及无法被回收）。它包含文件的基本属性，如文件ID、文件名、文件大小、创建时间、文件用途和文件元数据。

此外， `File`类还定义了一系列抽象方法，比较常用的有：

* 异步读取文件内容的 `read_contents`方法
* 将文件内容写入本地路径的 `write_contents_to`方法

以及一些辅助方法：

* 生成文件的字符串表示形式
* 转换为字典形式

在File类的内部，其主要有两个继承子类，一个是 `Local File`，一个是 `Remote File`。

以下是 `File` 基类的属性以及方法介绍：

| 属性       | 类型           | 描述                         

In [10]:
content

'# File 模块\n\n## 1. 模块简介\n\n文件管理模块提供了用于管理文件的一系列类，方便用户与Agent进行交互，其中包括 `File` 基类及其子类、`FileManager` 、`GlobalFileManagerHandler`以及与远程文件服务器交互的  `RemoteFileClient`。\n\n推荐使用  `GlobalFileManagerHandler`在事件循环开始时初始化 `FileManager`以及获取全局的 `FileManager`，之后只需通过这个全局的 `FileManager`对文件进行增、删、查等操作以及获取Agent产生的文件。\n\n!!! notes 注意\n\n    - **不推荐**用户自行操作 `File`类以免造成资源泄露。\n\n    - `FileManager`操作文件主要用于异步函数中，在同步函数中使用可能会无效。\n\n    - `FileManager`将作为此模块中生命周期最长的对象，它会在关闭时回收所有的持有对象（RemoteClient/temp local file），请不要随意关闭它。如果需要关闭，需要先停止对其中注册文件的使用。\n\n## 2. 核心类\n\n下面简单介绍 `File` 模块的核心类，详细接口请参考[API文档](../package/erniebot_agent/file.md)。\n\n### 2.1 File 基类\n\n`File` 类是文件管理模块的基础类，用于表示通用的文件对象（不建议自行创建 `File` 类以免无法被 `Agent`识别使用以及无法被回收）。它包含文件的基本属性，如文件ID、文件名、文件大小、创建时间、文件用途和文件元数据。\n\n此外， `File`类还定义了一系列抽象方法，比较常用的有：\n\n* 异步读取文件内容的 `read_contents`方法\n* 将文件内容写入本地路径的 `write_contents_to`方法\n\n以及一些辅助方法：\n\n* 生成文件的字符串表示形式\n* 转换为字典形式\n\n在File类的内部，其主要有两个继承子类，一个是 `Local File`，一个是 `Remote File`。\n\n以下是 `File` 基类的属性以及方法介绍：\n\n| 属性       | 类型 

In [8]:
from erniebot_agent.retrieval import BaizhongSearch

knowledge_base = BaizhongSearch(
    access_token="4ce50e3378f418d271c480c8ddfa818537071dbe", knowledge_base_name="my_db"
)
# knowledge_base.search(step_input, top_k=self.top_k, filters=None)
# knowledge_base.create_knowledge_base(knowledge_base_name='my_db')

BaizhongError: error message is request error: {"code":"MissingDateHeader","message":"Request must have a \"date\" or \"x-bce-date\" header.","requestId":"ac2563e6-d417-4aa3-8a9e-fc523a5dabc6"} error code is 404

In [35]:
import nbformat

from typing import List, Optional

from langchain_core.documents import Document

from langchain.document_loaders.base import BaseLoader
from langchain.document_loaders.helpers import detect_file_encodings


def open_and_concatenate_ipynb(ipynb_path, encoding):
    # 读取.ipynb文件
    with open(ipynb_path, "r", encoding=encoding) as f:
        notebook_content = nbformat.read(f, as_version=4)

    # 按顺序拼接Markdown文本和code单元
    concatenated_content = ""
    for cell in notebook_content["cells"]:
        if cell["cell_type"] == "markdown":
            concatenated_content += cell["source"] + "\n\n"
        elif cell["cell_type"] == "code":
            concatenated_content += cell["source"] + "\n\n"

    # 返回拼接后的内容
    return concatenated_content


class NotebookLoader(BaseLoader):
    def __init__(
        self,
        file_path: str,
        encoding: str = "utf-8",
        autodetect_encoding: bool = False,
    ):
        """Initialize with file path."""
        self.file_path = file_path
        self.encoding = encoding
        self.autodetect_encoding = autodetect_encoding

    def load(self) -> List[Document]:
        """Load from file path."""
        text = ""
        try:
            text = open_and_concatenate_ipynb(ipynb_path=self.file_path, encoding=self.encoding)
        except UnicodeDecodeError as e:
            if self.autodetect_encoding:
                detected_encodings = detect_file_encodings(self.file_path)
                for encoding in detected_encodings:
                    try:
                        with open(self.file_path, encoding=encoding.encoding) as f:
                            text = f.read()
                        break
                    except UnicodeDecodeError:
                        continue
            else:
                raise RuntimeError(f"Error loading {self.file_path}") from e
        except Exception as e:
            raise RuntimeError(f"Error loading {self.file_path}") from e

        metadata = {"source": self.file_path}
        return [Document(page_content=text, metadata=metadata)]

In [38]:
from langchain.text_splitter import CharacterTextSplitter
import nbformat


# 示例用法
ipynb_path = "/Users/tanzhehao/Documents/ERINE/ERNIE-Bot-SDK/docs/cookbooks/agent/chat_models.ipynb"  # 替换为你的.ipynb文件路径

document = NotebookLoader(ipynb_path, encoding="utf-8").load()
# Choose a splitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
# Split the text
split_chunks = text_splitter.split_documents(document)

# Process the split chunks
for chunk in split_chunks:
    # Do something with each chunk
    print(chunk)

Created a chunk of size 647, which is longer than the specified 500


page_content='# Chat Models大语言模型\n\n## 简介\n\n在Agent智能体中，大语言模型提供理解、决策、调度等能力，是最基础的大脑模块。因此，本文我们介绍如何上手使用大语言模型。\n\n简而言之，大语言模型基于深度学习技术，能够自动理解和生成人类自然语言文本的模型，广泛应用于文本生成、机器翻译、自动摘要、问答系统、情感分析等任务。\n\n目前市面上已经有很多大语言模型，比如OpenAI的ChatGPT、百度的文心一言等等。大语言模型通常具有大量参数，对运行机器的显存和算量有很高的要求。为了方便大家使用，这些大语言模型都有对外提供相应的调用接口。\n\n在`ERNIE Bot Agent`中，我们支持快速调用文心一言的多个模型，包括`ernie-3.5`、`ernie-turbo`、`ernie-4.0`和`ernie-longtext`。' metadata={'source': '/Users/tanzhehao/Documents/ERINE/ERNIE-Bot-SDK/docs/cookbooks/agent/chat_models.ipynb'}
page_content='在`ERNIE Bot Agent`中，我们支持快速调用文心一言的多个模型，包括`ernie-3.5`、`ernie-turbo`、`ernie-4.0`和`ernie-longtext`。\n\n| 模型名称 | 说明 | 功能 | 输入token数量上限 |\n|:--- | :--- | :--- | :--- |\n| ernie-3.5 | 文心大模型3.5版本。具备优秀的知识增强和内容生成能力，在文本创作、问答、推理和代码生成等方面表现出色。 | 对话补全，函数调用 | 3000 |\n| ernie-turbo | 文心大模型。相比ernie-3.5模型具备更快的响应速度和学习能力，API调用成本更低。 | 对话补全 |  3000 |\n| ernie-4.0 | 文心大模型4.0版本，具备目前系列模型中最优的理解和生成能力。 | 对话补全，函数调用 |  3000 |\n| ernie-longtext | 文心大模型。在ernie-3.5模型的基础上增强了对长对话上下文的支持，输入token数量上限为7000。 | 对话补全，函数调用

In [25]:
split_chunks

[Document(page_content='# Chat Models大语言模型\n\n## 简介\n\n在Agent智能体中，大语言模型提供理解、决策、调度等能力，是最基础的大脑模块。因此，本文我们介绍如何上手使用大语言模型。\n\n简而言之，大语言模型基于深度学习技术，能够自动理解和生成人类自然语言文本的模型，广泛应用于文本生成、机器翻译、自动摘要、问答系统、情感分析等任务。\n\n目前市面上已经有很多大语言模型，比如OpenAI的ChatGPT、百度的文心一言等等。大语言模型通常具有大量参数，对运行机器的显存和算量有很高的要求。为了方便大家使用，这些大语言模型都有对外提供相应的调用接口。\n\n在`ERNIE Bot Agent`中，我们支持快速调用文心一言的多个模型，包括`ernie-3.5`、`ernie-turbo`、`ernie-4.0`和`ernie-longtext`。'),
 Document(page_content='在`ERNIE Bot Agent`中，我们支持快速调用文心一言的多个模型，包括`ernie-3.5`、`ernie-turbo`、`ernie-4.0`和`ernie-longtext`。\n\n| 模型名称 | 说明 | 功能 | 输入token数量上限 |\n|:--- | :--- | :--- | :--- |\n| ernie-3.5 | 文心大模型3.5版本。具备优秀的知识增强和内容生成能力，在文本创作、问答、推理和代码生成等方面表现出色。 | 对话补全，函数调用 | 3000 |\n| ernie-turbo | 文心大模型。相比ernie-3.5模型具备更快的响应速度和学习能力，API调用成本更低。 | 对话补全 |  3000 |\n| ernie-4.0 | 文心大模型4.0版本，具备目前系列模型中最优的理解和生成能力。 | 对话补全，函数调用 |  3000 |\n| ernie-longtext | 文心大模型。在ernie-3.5模型的基础上增强了对长对话上下文的支持，输入token数量上限为7000。 | 对话补全，函数调用 |  7000 |\n\n## 上手使用文心一言'),
 Document(page_content='## 上手使用文心一言\n\n### 安装\n\n大家可以参考[官方

In [15]:
split_chunks

['# Chat Models大语言模型\n\n## 简介\n\n在Agent智能体中，大语言模型提供理解、决策、调度等能力，是最基础的大脑模块。因此，本文我们介绍如何上手使用大语言模型。\n\n简而言之，大语言模型基于深度学习技术，能够自动理解和生成人类自然语言文本的模型，广泛应用于文本生成、机器翻译、自动摘要、问答系统、情感分析等任务。\n\n目前市面上已经有很多大语言模型，比如OpenAI的ChatGPT、百度的文心一言等等。大语言模型通常具有大量参数，对运行机器的显存和算量有很高的要求。为了方便大家使用，这些大语言模型都有对外提供相应的调用接口。\n\n在`ERNIE Bot Agent`中，我们支持快速调用文心一言的多个模型，包括`ernie-3.5`、`ernie-turbo`、`ernie-4.0`和`ernie-longtext`。',
 '在`ERNIE Bot Agent`中，我们支持快速调用文心一言的多个模型，包括`ernie-3.5`、`ernie-turbo`、`ernie-4.0`和`ernie-longtext`。\n\n| 模型名称 | 说明 | 功能 | 输入token数量上限 |\n|:--- | :--- | :--- | :--- |\n| ernie-3.5 | 文心大模型3.5版本。具备优秀的知识增强和内容生成能力，在文本创作、问答、推理和代码生成等方面表现出色。 | 对话补全，函数调用 | 3000 |\n| ernie-turbo | 文心大模型。相比ernie-3.5模型具备更快的响应速度和学习能力，API调用成本更低。 | 对话补全 |  3000 |\n| ernie-4.0 | 文心大模型4.0版本，具备目前系列模型中最优的理解和生成能力。 | 对话补全，函数调用 |  3000 |\n| ernie-longtext | 文心大模型。在ernie-3.5模型的基础上增强了对长对话上下文的支持，输入token数量上限为7000。 | 对话补全，函数调用 |  7000 |\n\n## 上手使用文心一言',
 '## 上手使用文心一言\n\n### 安装\n\n大家可以参考[官方文档](https://github.com/PaddlePaddle/ERNIE-Bot-SDK)，进行安装ERNIE Bot Age

In [10]:
print(chunk)

function_message = FunctionMessage(
        name=name, content=json.dumps(result, ensure_ascii=False)
    )  # 构建FunctionMessage，封装函数的结果
    messages.append(function_message)

    ai_message = await model.chat(messages=messages, functions=functions)  # 将函数结果返回给模型，进行润色，得到最终输出
    print("final result:", ai_message.content)


In [4]:
print(chunk)

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Chat Models大语言模型\n",
    "\n",
    "## 简介\n",
    "\n",
    "在Agent智能体中，大语言模型提供理解、决策、调度等能力，是最基础的大脑模块。因此，本文我们介绍如何上手使用大语言模型。\n",
    "\n",
    "简而言之，大语言模型基于深度学习技术，能够自动理解和生成人类自然语言文本的模型，广泛应用于文本生成、机器翻译、自动摘要、问答系统、情感分析等任务。\n",
    "\n",
    "目前市面上已经有很多大语言模型，比如OpenAI的ChatGPT、百度的文心一言等等。大语言模型通常具有大量参数，对运行机器的显存和算量有很高的要求。为了方便大家使用，这些大语言模型都有对外提供相应的调用接口。\n",
    "\n",
    "在`ERNIE Bot Agent`中，我们支持快速调用文心一言的多个模型，包括`ernie-3.5`、`ernie-turbo`、`ernie-4.0`和`ernie-longtext`。\n",
    "\n",
    "| 模型名称 | 说明 | 功能 | 输入token数量上限 |\n",
    "|:--- | :--- | :--- | :--- |\n",
    "| ernie-3.5 | 文心大模型3.5版本。具备优秀的知识增强和内容生成能力，在文本创作、问答、推理和代码生成等方面表现出色。 | 对话补全，函数调用 | 3000 |\n",
    "| ernie-turbo | 文心大模型。相比ernie-3.5模型具备更快的响应速度和学习能力，API调用成本更低。 | 对话补全 |  3000 |\n",
    "| ernie-4.0 | 文心大模型4.0版本，具备目前系列模型中最优的理解和生成能力。 | 对话补全，函数调用 |  3000 |\n",
    "| ernie-longtext | 文心大模型。在ernie-3.5模型的基础上增强了对长对话上下文的支持，输入token数量上限为7000。 | 对话补全，函数调用 |  7000 |"
 

In [5]:
import requests
from bs4 import BeautifulSoup

url = "https://ernie-bot-agent.readthedocs.io/zh-cn/latest/quickstart/preparation/"
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Safari/605.1.15"
}
r = requests.get(url, headers=headers)
r.encoding = "utf-8"
html = r.text
soup = BeautifulSoup(html, "html.parser")

In [48]:
soup.find_all("h4", class_="doc")

[<h4 class="doc doc-heading" id="erniebot_agent.agents.Agent.__init__">
 <code class="highlight language-python"><span class="fm">__init__</span><span class="p">(</span><span class="n">llm</span><span class="p">,</span> <span class="n">tools</span><span class="p">,</span> <span class="o">*</span><span class="p">,</span> <span class="n">memory</span><span class="o">=</span><span class="kc">None</span><span class="p">,</span> <span class="n">system_message</span><span class="o">=</span><span class="kc">None</span><span class="p">,</span> <span class="n">callbacks</span><span class="o">=</span><span class="kc">None</span><span class="p">,</span> <span class="n">file_manager</span><span class="o">=</span><span class="kc">None</span><span class="p">,</span> <span class="n">plugins</span><span class="o">=</span><span class="kc">None</span><span class="p">)</span></code>
 <a class="headerlink" href="#erniebot_agent.agents.Agent.__init__" title="Permanent link">¶</a></h4>,
 <h4 class="doc doc-

In [6]:
print(soup)


<!DOCTYPE html>

<html class="no-js" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="provides an easy-to-use Chinese agent builder tool suite." name="description"/>
<link href="../agent/" rel="next"/>
<link href="../../img/favicon.png" rel="icon"/>
<meta content="mkdocs-1.5.3, mkdocs-material-9.5.3" name="generator"/>
<title>安装和鉴权 - ERNIE Bot Agent</title>
<link href="../../assets/stylesheets/main.50c56a3b.min.css" rel="stylesheet"/>
<link href="../../assets/stylesheets/palette.06af60db.min.css" rel="stylesheet"/>
<link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
<link href="https://fonts.googleapis.com/css?family=Noto+Serif+Simplified+Chinese:300,300i,400,400i,700,700i%7CNoto+Serif+Simplified+Chinese:400,400i,700,700i&amp;display=fallback" rel="stylesheet"/>
<style>:root{--md-text-font:"Noto Serif Simplified Chinese";--md-code-font:"Noto Serif Simplified Chinese"}</style>
<link href="